# Make clean data

In [3]:
import pandas as pd

bubbleList = []
countingList = []
mergeList = []
quickList = []
readCSVList = []
pcList = ["6", "16", "21", "23"]

quickList.append(pd.read_csv("SummerResults/quickSort-400runs-PC16.csv"))
quickList[0].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]

bubbleList.append(pd.read_csv("SummerResults/bubbleSort-400runs-PC16.csv"))
countingList[0].columns=["Test name","PKG (J)","CPU (J)","GPU (J)","DRAM (J)","Temp (C)","Time (ms)"]


In [4]:
def removeNonsense(df):
    df = df[df["PKG (J)"]>=0]
    df = df[df["CPU (J)"]>=0]
    df = df[df["Time (ms)"]>=0]
    return df

In [5]:
def makeinputColumn(df):
    l = []
    for i in range(len(df)):
        l.append(df.iloc[i]["Test name"].split("-")[0])

    df["Input case"] = l

def makeSizecolumn(df):
    l = []
    for i in range(len(df)):
        l.append(int(df.iloc[i]["Test name"].split("-")[1]))

    df["Size"] = l

In [6]:
def makeinputColumn_readCSV(df):
    l = []
    for i in range(len(df)):
        l.append(df.iloc[i]["Test name"].split("-")[1].lower())
    df["Input case"] = l

def makeSizecolumn_readCSV(df):
    l = []
    for i in range(len(df)):
        l.append(int(df.iloc[i]["Test name"].split("-")[-1]))

    df["Size"] = l

In [7]:
def removeOutliers(df, type_, size, col):
    q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
    q3 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.75)
    IQR = q3 - q1 
    LF = q1 - 1.5 * IQR
    UF = q3 + 1.5 * IQR

    df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]
    df2 = df1[df1["Input case"] == type_][df1["Size"]==size][df1[col] >= LF]
    return df2[col]


In [8]:
def CleanDataframe(df, DataOrBase):
    if DataOrBase == "data":
        cases = ["best", "random", "worst"]
    else:
        cases = ["random", "countingrandom", "countingbest", "countingworst", "mergeworst", "quickbest", "reversesorted", "sorted"]

    cleanList = []
    for case in cases:
        if case != "random" and case != "countingrandom":
            for size in [25000,50000,75000,100000,200000,300000,400000,500000,600000,700000,800000,900000,1000000]:
                cleanPortion = pd.DataFrame({"PKG (J)":[]})
                for col in ["PKG (J)", "CPU (J)", "DRAM (J)","Time (ms)", "Temp (C)"]:
                    cleanPortion[col] = removeOutliers(df, case, size, col)
                
                cleanPortion["Input case"] = [case for _ in range(len(cleanPortion))]
                cleanPortion["Size"] = [int(size) for _ in range(len(cleanPortion))]

                cleanList.append(cleanPortion)
        else:
            for i in range(1,11):
                for size in [25000,50000,75000,100000,200000,300000,400000,500000,600000,700000,800000,900000,1000000]:
                    cleanPortion = pd.DataFrame({"PKG (J)":[]})
                    for col in ["PKG (J)", "CPU (J)", "DRAM (J)","Time (ms)", "Temp (C)"]:
                        cleanPortion[col] = removeOutliers(df, case+str(i), size, col)
                
                    cleanPortion["Input case"] = [case+str(i) for _ in range(len(cleanPortion))]
                    cleanPortion["Size"] = [int(size) for _ in range(len(cleanPortion))]
                    cleanList.append(cleanPortion)
    cleanBubble = pd.concat(cleanList)
    return cleanBubble

In [9]:
def prepareDataframe(df, DataOrBase):
    df = removeNonsense(df)

    if DataOrBase == "data":
        makeinputColumn(df)
        makeSizecolumn(df)
    else:
        makeinputColumn_readCSV(df)
        makeSizecolumn_readCSV(df)    

    return df


In [10]:
def CleanCSV (df, DataOrBase, name, pcNO):
    ready = prepareDataframe(df, DataOrBase)
    clean = CleanDataframe(ready, DataOrBase).dropna().reset_index(drop=True)
    print(clean)
    clean.to_csv("FinalAnalysis\\CleanData\\"+name+"Clean-PC"+pcNO+".csv", sep=",", index=False)

In [11]:
# for i in range(len(bubbleList)):
#     CleanCSV(bubbleList[i], "data","Bubble",pcList[i])

# for i in range(len(bubbleList)):
#     CleanCSV(countingList[i], "data","Counting",pcList[i])

# for i in range(len(bubbleList)):
#     CleanCSV(mergeList[i], "data","Merge",pcList[i])

# for i in range(len(bubbleList)):
#     CleanCSV(quickList[i], "data","Quick",pcList[i])

# for i in range(len(bubbleList)):
#     CleanCSV(readCSVList[i], "base","readCSV",pcList[i])


CleanCSV(quickList[0], "data","Qucik-400runs",pcList[1])
CleanCSV(countingList[0], "data","Counting-400runs",pcList[1])

C:\Users\langk\AppData\Local\Temp\ipykernel_14068\2727353117.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
C:\Users\langk\AppData\Local\Temp\ipykernel_14068\2727353117.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q3 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.75)
C:\Users\langk\AppData\Local\Temp\ipykernel_14068\2727353117.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]
C:\Users\langk\AppData\Local\Temp\ipykernel_14068\2727353117.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
C:\Users\langk\AppData\Local\Temp\ipykernel_14068\2727353117.py:3: UserWarning: Boolean Series key will be reindexed to match 

        PKG (J)     CPU (J)   DRAM (J)  Time (ms)  Temp (C) Input case  \
0      4.396484    3.866089   0.213867    424.846      64.0       best   
1      4.326294    3.802795   0.212158    418.592      64.0       best   
2      4.566711    4.002136   0.215454    470.242      64.0       best   
3      4.493408    3.941650   0.214966    455.232      64.0       best   
4      4.564880    3.991516   0.213379    497.503      64.0       best   
..          ...         ...        ...        ...       ...        ...   
725  320.330994  277.220337   9.595825  46171.800      61.0   random10   
726  344.648315  298.666748  10.122620  49438.900      55.0   random10   
727  359.739807  311.416016  10.789001  51801.400      61.0   random10   
728  369.317993  319.709045  11.041138  53394.700      55.0   random10   
729  358.772217  310.618164  10.776367  51770.400      55.0   random10   

          Size  
0      25000.0  
1      25000.0  
2      25000.0  
3      25000.0  
4      25000.0  
..       

C:\Users\langk\AppData\Local\Temp\ipykernel_14068\2727353117.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
C:\Users\langk\AppData\Local\Temp\ipykernel_14068\2727353117.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q3 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.75)
C:\Users\langk\AppData\Local\Temp\ipykernel_14068\2727353117.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]
C:\Users\langk\AppData\Local\Temp\ipykernel_14068\2727353117.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q1 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.25)
C:\Users\langk\AppData\Local\Temp\ipykernel_14068\2727353117.py:3: UserWarning: Boolean Series key will be reindexed to match 

        PKG (J)     CPU (J)    DRAM (J)   Time (ms)  Temp (C) Input case  \
0      2.893311    2.514954    0.166016     286.242      57.0       best   
1      2.569702    2.237183    0.142822     271.777      57.0       best   
2      3.000427    2.612549    0.178101     295.577      57.0       best   
3      2.849365    2.477051    0.157349     293.878      57.0       best   
4      2.866882    2.465088    0.181396     319.299      57.0       best   
..          ...         ...         ...         ...       ...        ...   
701  493.700745  330.640930  111.372681  111212.000      53.0      worst   
702  493.633667  330.534607  111.522034  111266.000      53.0      worst   
703  493.404846  330.862610  111.068359  110870.000      53.0      worst   
704  493.722107  330.744995  111.461060  111168.000      53.0      worst   
705  494.318481  331.167358  111.460938  111336.000      53.0      worst   

          Size  
0      25000.0  
1      25000.0  
2      25000.0  
3      25000.0  
4 

C:\Users\langk\AppData\Local\Temp\ipykernel_14068\2727353117.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  q3 = df[df["Input case"] == type_][df["Size"]==size][col].quantile(q=0.75)
C:\Users\langk\AppData\Local\Temp\ipykernel_14068\2727353117.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df[df["Input case"] == type_][df["Size"]==size][df[col] <= UF]


In [12]:
#CleanCSV(bubbleList[0], "data","Bubble",pcList[0])
